In [35]:
from sim import NBodySimulation, animate3d, generateDisk3Dv3
import numpy as np
import json

In [47]:


def gen_params():
    return {
        'nbStars': np.random.randint(100, 1000),
        'radius': np.random.uniform(0.5, 2.5),
        'Mass': np.random.uniform(1.0, 3.0),
        'zOffsetMax': np.random.uniform(0, 0.5),
        'gravityCst': 1.0,
        'distribution': 'hernquist',
        'offset': [np.random.uniform(-1, 1), np.random.uniform(-1, 1), np.random.uniform(-1, 1)],
        'initial_vel': [np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1)],
        'clockwise': np.random.choice([1, 0]),
        'angle': [np.random.uniform(-1, 1)*2*np.pi, np.random.uniform(-1, 1)*2*np.pi, np.random.uniform(-1, 1)*2*np.pi]
    }

def generate_scene_2gals():
    params1 = gen_params()
    params2 = gen_params()
    print(params1)
    print(params2)

    particles1 = generateDisk3Dv3(**params1)
    particles2 = generateDisk3Dv3(**params2)

    t_end = 10.0
    dt = 0.01
    softening = 0.1
    G = 1.0

    particles = particles1 + particles2
    sim = NBodySimulation(particles, G, softening, dt)

    pos, vel, acc, KE, PE, t_all, masses = sim.run(t_end=t_end, plot_real_time=False, save_states=True)

    frames = []
    for i in range(len(pos)):
        frames.append({
            'frame': i,
            'pos': pos[i],
            'vel': vel[i],
            'acc': acc[i]
        })

    final_json = {
        'galaxy1_params': params1,
        'galaxy2_params': params2,
        'dt' : dt,
        'softening' : softening,
        'G' : G,
        't_end' : t_end,
        'masses' : masses,
        'frames': frames
    }

    return final_json


In [48]:
json_test = generate_scene_2gals()


{'nbStars': 706, 'radius': 0.7386780411191578, 'Mass': 2.6776410070855037, 'zOffsetMax': 0.2562402292968483, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [0.9292163498656545, 0.786145495921933, -0.8203354948537347], 'initial_vel': [-0.04835851303085255, 0.07372970426092315, -0.010102343150972984], 'clockwise': 0, 'angle': [5.218288418639533, 0.11874579463687168, -2.2705435294061505]}
{'nbStars': 971, 'radius': 1.6728951357472, 'Mass': 2.3419080633022027, 'zOffsetMax': 0.192480423861686, 'gravityCst': 1.0, 'distribution': 'hernquist', 'offset': [-0.22252414830821032, -0.3989387585214399, -0.8786233850113603], 'initial_vel': [-0.07367466116301435, 0.08673275778394252, 0.020355692126402716], 'clockwise': 1, 'angle': [1.0887892124493455, 5.415642531539282, -1.50228113020012]}


100%|██████████| 1000/1000 [02:31<00:00,  6.59it/s]


IndexError: index 1001 is out of bounds for axis 0 with size 1001

In [ ]:
print(json.dumps(json_test, indent=4))